# 🎬 LOCOMOT.IO Automatic Clip Generator

Generates high-quality promotional clips with full game rendering.

**Features:**
- Full game-accurate rendering (train cars, glow, particles, grid)
- AI trains with neural network brains
- Smart clip selection: only keeps kills where train keeps strafing (not coiling)
- Excitement scoring and ranking
- Exports as MP4 videos

In [ ]:
# Install dependencies
!pip install -q numpy pillow imageio imageio-ffmpeg

In [ ]:
import numpy as np
from PIL import Image, ImageDraw, ImageFilter, ImageFont
import imageio
from collections import deque
import random
import math
from dataclasses import dataclass, field
from typing import List, Tuple, Optional, Dict
import time
from IPython.display import HTML, display, Video
import base64
import os

In [ ]:
# === GAME CONSTANTS (matching actual game) ===
WORLD_W, WORLD_H = 100, 75
GRID = 16
CANVAS_W, CANVAS_H = 640, 480
FPS = 30
CLIP_SECONDS = 8
BUFFER_FRAMES = FPS * CLIP_SECONDS

# Train colors from the actual game
TRAIN_COLORS = [
    (255, 0, 255),   # Magenta (neural)
    (0, 170, 255),   # Blue
    (255, 85, 85),   # Red
    (255, 200, 0),   # Gold
    (0, 255, 200),   # Cyan
    (255, 100, 200), # Pink
    (180, 100, 255), # Purple
    (100, 255, 100), # Light green
    (255, 150, 50),  # Orange
    (150, 255, 255), # Light cyan
]

# Gun types
GUN_TYPES = [
    {'name': 'PISTOL', 'color': (150, 150, 150)},
    {'name': 'SMG', 'color': (100, 200, 255)},
    {'name': 'SHOTGUN', 'color': (255, 150, 50)},
    {'name': 'SNIPER', 'color': (255, 50, 50)},
    {'name': 'LASER', 'color': (255, 0, 255)},
]

In [ ]:
# === SEGMENT AND TRAIN ===
@dataclass
class Segment:
    x: int
    y: int
    hp: int = 100
    max_hp: int = 100
    gun: Optional[Dict] = None

@dataclass 
class Train:
    id: int
    name: str
    segments: List[Segment] = field(default_factory=list)
    color: Tuple[int, int, int] = (255, 0, 255)
    direction: Tuple[int, int] = (1, 0)
    alive: bool = True
    kills: int = 0
    brain: np.ndarray = None
    brain2: np.ndarray = None
    
    # Movement history for coil detection
    direction_history: List[Tuple[int, int]] = field(default_factory=list)
    position_history: List[Tuple[int, int]] = field(default_factory=list)
    
    def __post_init__(self):
        if self.brain is None:
            self.brain = np.random.randn(12, 16) * 0.5
            self.brain2 = np.random.randn(16, 3) * 0.5
    
    @property
    def head(self):
        return self.segments[0] if self.segments else None
    
    @property
    def length(self):
        return len(self.segments)
    
    def get_vision(self, game) -> np.ndarray:
        """12-input vision: 8 directions + food direction + nearest enemy direction"""
        if not self.head:
            return np.zeros(12)
        
        vision = []
        directions = [(0,-1), (1,-1), (1,0), (1,1), (0,1), (-1,1), (-1,0), (-1,-1)]
        
        for dx, dy in directions:
            dist = 0
            x, y = self.head.x, self.head.y
            found = False
            
            while dist < 15 and not found:
                x = (x + dx) % WORLD_W
                y = (y + dy) % WORLD_H
                dist += 1
                
                for train in game.trains:
                    if train.id != self.id and train.alive:
                        for seg in train.segments:
                            if seg.x == x and seg.y == y:
                                found = True
                                break
                    if found:
                        break
            
            vision.append(1.0 - dist / 15.0)
        
        # Add food direction
        if game.food:
            nearest_food = min(game.food, key=lambda f: abs(f[0]-self.head.x) + abs(f[1]-self.head.y))
            fx = (nearest_food[0] - self.head.x) / WORLD_W
            fy = (nearest_food[1] - self.head.y) / WORLD_H
            vision.extend([fx, fy])
        else:
            vision.extend([0, 0])
        
        # Add current direction
        vision.extend([self.direction[0], self.direction[1]])
        
        return np.array(vision)
    
    def think(self, game) -> int:
        """Returns 0=left, 1=straight, 2=right"""
        vision = self.get_vision(game)
        hidden = np.tanh(vision @ self.brain)
        output = hidden @ self.brain2
        return int(np.argmax(output))
    
    def turn(self, decision: int):
        dx, dy = self.direction
        if decision == 0:  # Left
            self.direction = (dy, -dx)
        elif decision == 2:  # Right
            self.direction = (-dy, dx)
        
        # Track direction history
        self.direction_history.append(self.direction)
        if len(self.direction_history) > 60:  # 2 seconds
            self.direction_history.pop(0)
    
    def move(self):
        if not self.alive or not self.head:
            return
        
        new_x = (self.head.x + self.direction[0]) % WORLD_W
        new_y = (self.head.y + self.direction[1]) % WORLD_H
        
        # Track position
        self.position_history.append((new_x, new_y))
        if len(self.position_history) > 60:
            self.position_history.pop(0)
        
        for i in range(len(self.segments) - 1, 0, -1):
            self.segments[i].x = self.segments[i-1].x
            self.segments[i].y = self.segments[i-1].y
        
        self.head.x = new_x
        self.head.y = new_y
    
    def grow(self, amount: int = 1):
        for _ in range(amount):
            tail = self.segments[-1]
            gun = random.choice(GUN_TYPES) if random.random() < 0.3 else None
            self.segments.append(Segment(tail.x, tail.y, gun=gun))
    
    def is_coiling(self) -> bool:
        """Detect if train is coiling (turning in circles) vs strafing (moving forward)"""
        if len(self.direction_history) < 30:
            return False
        
        # Count direction changes in last second
        recent = self.direction_history[-30:]
        changes = sum(1 for i in range(1, len(recent)) if recent[i] != recent[i-1])
        
        # If changing direction more than 10 times per second = coiling
        return changes > 10
    
    def is_strafing_forward(self) -> bool:
        """Check if train is moving forward with purpose (not spinning)"""
        if len(self.position_history) < 30:
            return True
        
        # Calculate net displacement over last second
        start = self.position_history[0]
        end = self.position_history[-1]
        
        # Account for wrapping
        dx = min(abs(end[0] - start[0]), WORLD_W - abs(end[0] - start[0]))
        dy = min(abs(end[1] - start[1]), WORLD_H - abs(end[1] - start[1]))
        displacement = math.sqrt(dx*dx + dy*dy)
        
        # Should have moved at least 10 cells in a second if strafing
        return displacement > 8

In [ ]:
# === EVENT TRACKING WITH STRAFE DETECTION ===
@dataclass
class ExcitingEvent:
    frame: int
    event_type: str
    score: float
    details: Dict = field(default_factory=dict)
    killer_id: Optional[int] = None
    is_valid: bool = True  # Will be set false if killer coils after

class ExcitementTracker:
    def __init__(self):
        self.events: List[ExcitingEvent] = []
        self.kill_times: Dict[int, List[int]] = {}
        self.leader_id: Optional[int] = None
        self.pending_validation: List[Tuple[ExcitingEvent, int]] = []  # (event, validation_frame)
    
    def on_kill(self, frame: int, killer: Train, victim: Train):
        if killer.id not in self.kill_times:
            self.kill_times[killer.id] = []
        self.kill_times[killer.id].append(frame)
        
        recent = [f for f in self.kill_times[killer.id] if frame - f < FPS * 3]
        streak = len(recent)
        
        length_diff = killer.length - victim.length
        score = 50 + (streak - 1) * 40 + max(0, -length_diff) * 5
        
        event_type = 'streak' if streak >= 2 else 'kill'
        event = ExcitingEvent(
            frame=frame,
            event_type=event_type,
            score=score,
            details={'killer': killer.name, 'victim': victim.name, 'streak': streak, 'killer_len': killer.length},
            killer_id=killer.id
        )
        self.events.append(event)
        
        # Schedule validation 1.5 seconds later
        self.pending_validation.append((event, frame + int(FPS * 1.5)))
        
        print(f"  💀 KILL: {killer.name} → {victim.name} (score: {score})")
    
    def validate_events(self, current_frame: int, trains: List[Train]):
        """Check if killers are still strafing (not coiling) after their kills"""
        still_pending = []
        
        for event, validation_frame in self.pending_validation:
            if current_frame >= validation_frame:
                # Find the killer
                killer = next((t for t in trains if t.id == event.killer_id), None)
                if killer and killer.alive:
                    if killer.is_coiling() or not killer.is_strafing_forward():
                        event.is_valid = False
                        print(f"  ❌ Invalidated: {event.details['killer']} coiled after kill")
                    else:
                        print(f"  ✅ Validated: {event.details['killer']} kept strafing")
            else:
                still_pending.append((event, validation_frame))
        
        self.pending_validation = still_pending
    
    def on_leader_change(self, frame: int, new_leader: Train):
        if self.leader_id != new_leader.id:
            self.events.append(ExcitingEvent(
                frame=frame,
                event_type='leader_change',
                score=35,
                details={'new_leader': new_leader.name, 'length': new_leader.length}
            ))
            self.leader_id = new_leader.id
    
    def get_best_moments(self, n: int = 5) -> List[ExcitingEvent]:
        # Only return valid events (where killer kept strafing)
        valid_events = [e for e in self.events if e.is_valid]
        return sorted(valid_events, key=lambda e: e.score, reverse=True)[:n]

In [ ]:
# === HEADLESS GAME ===
class HeadlessGame:
    def __init__(self, num_trains: int = 10):
        self.trains: List[Train] = []
        self.food: List[Tuple[int, int]] = []
        self.particles: List[Dict] = []
        self.bullets: List[Dict] = []
        self.frame = 0
        self.tracker = ExcitementTracker()
        
        names = ['ShadowX', 'NightWolf', 'Blaze', 'IceKing', 'Thunder', 
                 'Phoenix', 'Storm', 'Ninja', 'Ghost', 'Iron', 'Viper', 'Reaper']
        
        for i in range(num_trains):
            x = random.randint(15, WORLD_W - 15)
            y = random.randint(15, WORLD_H - 15)
            train = Train(
                id=i,
                name=names[i % len(names)] + str(random.randint(10, 99)),
                segments=[Segment(x-j, y, gun=random.choice(GUN_TYPES) if j > 0 and random.random() < 0.4 else None) for j in range(5)],
                color=TRAIN_COLORS[i % len(TRAIN_COLORS)],
                direction=random.choice([(1,0), (-1,0), (0,1), (0,-1)])
            )
            self.trains.append(train)
        
        for _ in range(50):
            self.spawn_food()
    
    def spawn_food(self):
        self.food.append((random.randint(0, WORLD_W-1), random.randint(0, WORLD_H-1)))
    
    def respawn_train(self, train: Train):
        x = random.randint(15, WORLD_W - 15)
        y = random.randint(15, WORLD_H - 15)
        train.segments = [Segment(x-j, y) for j in range(4)]
        train.direction = random.choice([(1,0), (-1,0), (0,1), (0,-1)])
        train.alive = True
        train.direction_history.clear()
        train.position_history.clear()
    
    def step(self):
        self.frame += 1
        alive_trains = [t for t in self.trains if t.alive]
        
        # Validate pending kill events
        self.tracker.validate_events(self.frame, self.trains)
        
        # AI + movement
        for train in alive_trains:
            decision = train.think(self)
            train.turn(decision)
            train.move()
        
        # Food
        for train in alive_trains:
            head = train.head
            for i, (fx, fy) in enumerate(self.food):
                if head.x == fx and head.y == fy:
                    train.grow(2)
                    self.food.pop(i)
                    self.spawn_food()
                    break
        
        # Shooting
        for train in alive_trains:
            if random.random() < 0.1:  # 10% chance to shoot per frame
                for seg in train.segments[1:]:
                    if seg.gun and random.random() < 0.3:
                        # Fire bullet in train's direction
                        self.bullets.append({
                            'x': seg.x * GRID + GRID//2,
                            'y': seg.y * GRID + GRID//2,
                            'vx': train.direction[0] * 8,
                            'vy': train.direction[1] * 8,
                            'color': seg.gun['color'],
                            'owner_id': train.id,
                            'life': 30
                        })
        
        # Update bullets
        new_bullets = []
        for b in self.bullets:
            b['x'] += b['vx']
            b['y'] += b['vy']
            b['life'] -= 1
            if b['life'] > 0 and 0 <= b['x'] < WORLD_W*GRID and 0 <= b['y'] < WORLD_H*GRID:
                new_bullets.append(b)
        self.bullets = new_bullets
        
        # Collisions
        for train in alive_trains:
            head = train.head
            for other in alive_trains:
                if other.id == train.id:
                    continue
                for seg in other.segments:
                    if head.x == seg.x and head.y == seg.y:
                        if train.length > other.length:
                            train.kills += 1
                            train.grow(other.length // 2)
                            self.add_particles(other)
                            self.tracker.on_kill(self.frame, train, other)
                            other.alive = False
                        elif other.length > train.length:
                            other.kills += 1
                            other.grow(train.length // 2)
                            self.add_particles(train)
                            self.tracker.on_kill(self.frame, other, train)
                            train.alive = False
                        else:
                            self.add_particles(train)
                            self.add_particles(other)
                            train.alive = False
                            other.alive = False
                        break
                if not train.alive:
                    break
        
        # Respawn
        for train in self.trains:
            if not train.alive:
                self.respawn_train(train)
        
        # Leader tracking
        if alive_trains:
            leader = max(alive_trains, key=lambda t: t.length)
            self.tracker.on_leader_change(self.frame, leader)
        
        # Particles
        self.particles = [p for p in self.particles if p['life'] > 0]
        for p in self.particles:
            p['x'] += p['vx']
            p['y'] += p['vy']
            p['vy'] += 0.2  # gravity
            p['life'] -= 1
    
    def add_particles(self, train: Train):
        for seg in train.segments:
            for _ in range(5):
                self.particles.append({
                    'x': seg.x * GRID + GRID//2,
                    'y': seg.y * GRID + GRID//2,
                    'vx': random.uniform(-4, 4),
                    'vy': random.uniform(-6, 2),
                    'life': random.randint(15, 30),
                    'color': train.color,
                    'size': random.randint(2, 5)
                })

In [ ]:
# === FULL GAME RENDERER ===
class GameRenderer:
    def __init__(self):
        pass
    
    def blend_color(self, c1, c2, t):
        return tuple(int(c1[i] + (c2[i] - c1[i]) * t) for i in range(3))
    
    def dim_color(self, color, factor=0.5):
        return tuple(int(c * factor) for c in color)
    
    def render_frame(self, game: HeadlessGame, camera_target: Optional[Train] = None) -> Image.Image:
        # Create base image
        img = Image.new('RGB', (CANVAS_W, CANVAS_H), (17, 17, 17))
        draw = ImageDraw.Draw(img)
        
        # Camera
        if camera_target and camera_target.head:
            cam_x = camera_target.head.x * GRID - CANVAS_W // 2
            cam_y = camera_target.head.y * GRID - CANVAS_H // 2
        else:
            cam_x = (WORLD_W * GRID - CANVAS_W) // 2
            cam_y = (WORLD_H * GRID - CANVAS_H) // 2
        
        # Grid with glow effect
        for x in range(0, WORLD_W * GRID, GRID):
            sx = x - cam_x
            if -GRID <= sx < CANVAS_W + GRID:
                draw.line([(sx, 0), (sx, CANVAS_H)], fill=(30, 35, 30), width=1)
        for y in range(0, WORLD_H * GRID, GRID):
            sy = y - cam_y
            if -GRID <= sy < CANVAS_H + GRID:
                draw.line([(0, sy), (CANVAS_W, sy)], fill=(30, 35, 30), width=1)
        
        # Food (yellow orbs with glow)
        for fx, fy in game.food:
            sx = fx * GRID - cam_x + GRID // 2
            sy = fy * GRID - cam_y + GRID // 2
            if -10 <= sx < CANVAS_W + 10 and -10 <= sy < CANVAS_H + 10:
                # Outer glow
                for r in range(6, 2, -1):
                    alpha = (6 - r) / 4
                    glow_color = (int(100 * alpha), int(100 * alpha), 0)
                    draw.ellipse([sx-r, sy-r, sx+r, sy+r], fill=glow_color)
                # Core
                draw.ellipse([sx-3, sy-3, sx+3, sy+3], fill=(255, 255, 0))
        
        # Trains
        for train in sorted(game.trains, key=lambda t: t.length):  # Draw smaller trains first
            if not train.alive:
                continue
            
            for i, seg in enumerate(train.segments):
                sx = seg.x * GRID - cam_x
                sy = seg.y * GRID - cam_y
                
                if -GRID*2 <= sx < CANVAS_W + GRID*2 and -GRID*2 <= sy < CANVAS_H + GRID*2:
                    is_head = (i == 0)
                    
                    if is_head:
                        # Head - brighter, slightly larger
                        color = train.color
                        size = GRID - 2
                        # Glow effect for head
                        glow_color = self.dim_color(color, 0.3)
                        draw.ellipse([sx-2, sy-2, sx+GRID+2, sy+GRID+2], fill=glow_color)
                    else:
                        # Body segments
                        color = self.dim_color(train.color, 0.6)
                        size = GRID - 4
                    
                    offset = (GRID - size) // 2
                    
                    # Draw segment
                    draw.rectangle(
                        [sx + offset, sy + offset, sx + offset + size, sy + offset + size],
                        fill=color,
                        outline=self.dim_color(color, 0.3)
                    )
                    
                    # Gun turret on segment
                    if seg.gun and not is_head:
                        gun_color = seg.gun['color']
                        cx, cy = sx + GRID//2, sy + GRID//2
                        # Turret base
                        draw.ellipse([cx-3, cy-3, cx+3, cy+3], fill=gun_color)
                        # Barrel
                        bx = cx + train.direction[0] * 5
                        by = cy + train.direction[1] * 5
                        draw.line([(cx, cy), (bx, by)], fill=gun_color, width=2)
                    
                    # Health bar for head
                    if is_head:
                        hp_pct = seg.hp / seg.max_hp
                        bar_w = GRID - 2
                        draw.rectangle([sx+1, sy-4, sx+1+bar_w, sy-2], fill=(50, 0, 0))
                        draw.rectangle([sx+1, sy-4, sx+1+int(bar_w*hp_pct), sy-2], fill=(0, 255, 0))
        
        # Bullets
        for b in game.bullets:
            bx = b['x'] - cam_x
            by = b['y'] - cam_y
            if 0 <= bx < CANVAS_W and 0 <= by < CANVAS_H:
                # Trail
                trail_x = bx - b['vx'] * 2
                trail_y = by - b['vy'] * 2
                draw.line([(trail_x, trail_y), (bx, by)], fill=self.dim_color(b['color'], 0.5), width=2)
                # Bullet
                draw.ellipse([bx-2, by-2, bx+2, by+2], fill=b['color'])
        
        # Particles
        for p in game.particles:
            px = p['x'] - cam_x
            py = p['y'] - cam_y
            if 0 <= px < CANVAS_W and 0 <= py < CANVAS_H:
                alpha = p['life'] / 30
                color = tuple(int(c * alpha) for c in p['color'])
                s = p['size']
                draw.ellipse([px-s, py-s, px+s, py+s], fill=color)
        
        # Leaderboard
        sorted_trains = sorted(game.trains, key=lambda t: t.length, reverse=True)[:5]
        
        # Leaderboard background
        draw.rectangle([CANVAS_W-145, 5, CANVAS_W-5, 5 + len(sorted_trains)*18 + 10], 
                      fill=(0, 0, 0, 180), outline=(0, 100, 0))
        
        y_pos = 12
        for i, train in enumerate(sorted_trains):
            prefix = "👑" if i == 0 else f"{i+1}."
            text = f"{prefix} {train.name[:8]}: {train.length}"
            draw.text((CANVAS_W - 140, y_pos), text, fill=train.color)
            y_pos += 18
        
        return img
    
    def render_clip(self, frames: List[Image.Image], filename: str):
        print(f"  Rendering {len(frames)} frames to {filename}...")
        with imageio.get_writer(filename, fps=FPS, quality=8, macro_block_size=1) as writer:
            for frame in frames:
                writer.append_data(np.array(frame))
        print(f"  ✅ Saved: {filename}")

In [ ]:
# === CLIP EXTRACTOR ===
class ClipExtractor:
    def __init__(self, game: HeadlessGame, renderer: GameRenderer):
        self.game = game
        self.renderer = renderer
    
    def run_simulation(self, total_frames: int = 4500):
        """Run simulation and collect frames"""
        print(f"🎮 Simulating {total_frames/FPS:.0f} seconds of gameplay...")
        print(f"   (Only clips where trains keep strafing will be kept)\n")
        
        all_frames = {}
        
        for i in range(total_frames):
            self.game.step()
            
            # Find leader for camera
            alive = [t for t in self.game.trains if t.alive]
            leader = max(alive, key=lambda t: t.length) if alive else None
            
            # Render
            frame = self.renderer.render_frame(self.game, camera_target=leader)
            all_frames[self.game.frame] = frame
            
            if i > 0 and i % 900 == 0:
                print(f"  {i/FPS:.0f}s - {len(self.game.tracker.events)} events found")
        
        # Final validation pass
        for _ in range(FPS * 2):
            self.game.step()
            self.game.tracker.validate_events(self.game.frame, self.game.trains)
        
        valid_events = [e for e in self.game.tracker.events if e.is_valid]
        print(f"\n✅ Done! {len(valid_events)} valid clips (trains kept strafing)")
        print(f"   {len(self.game.tracker.events) - len(valid_events)} rejected (trains coiled)")
        
        return all_frames
    
    def extract_clips(self, all_frames: Dict, num_clips: int = 5) -> List[Dict]:
        best_events = self.game.tracker.get_best_moments(num_clips)
        
        print(f"\n🎬 Extracting top {len(best_events)} clips...")
        
        clips = []
        
        for i, event in enumerate(best_events):
            start = max(1, event.frame - FPS * 3)
            end = min(max(all_frames.keys()), event.frame + FPS * 4)
            
            clip_frames = [all_frames[f] for f in range(start, end) if f in all_frames]
            
            if len(clip_frames) > FPS * 2:
                clips.append({'event': event, 'frames': clip_frames, 'rank': i + 1})
                print(f"  #{i+1} {event.event_type.upper()} - {event.details['killer']} (score: {event.score:.0f})")
        
        return clips
    
    def save_clips(self, clips: List[Dict], output_dir: str = 'clips'):
        os.makedirs(output_dir, exist_ok=True)
        print(f"\n💾 Rendering {len(clips)} clips...")
        
        saved = []
        for clip in clips:
            e = clip['event']
            filename = f"{output_dir}/clip_{clip['rank']:02d}_{e.event_type}_{int(e.score)}pts.mp4"
            self.renderer.render_clip(clip['frames'], filename)
            saved.append(filename)
        
        return saved

In [ ]:
# === RUN THE PIPELINE ===
print("🚂 LOCOMOT.IO Clip Generator")
print("=" * 50)
print("Generating clips where trains STRAFE, not coil\n")

# Initialize
game = HeadlessGame(num_trains=10)
renderer = GameRenderer()
extractor = ClipExtractor(game, renderer)

# Run simulation (2.5 minutes of gameplay)
all_frames = extractor.run_simulation(total_frames=FPS * 150)

# Extract best clips
clips = extractor.extract_clips(all_frames, num_clips=5)

# Save clips
saved_files = extractor.save_clips(clips)

print(f"\n🎉 Generated {len(saved_files)} premium clips!")

In [ ]:
# === PREVIEW CLIPS ===
print("\n📺 Preview clips:")
for i, filename in enumerate(saved_files[:3]):
    print(f"\nClip #{i+1}: {filename}")
    display(Video(filename, embed=True, width=640))

In [ ]:
# === DOWNLOAD ALL ===
from google.colab import files
import zipfile

zip_name = 'locomotio_ad_clips.zip'
with zipfile.ZipFile(zip_name, 'w') as zf:
    for f in saved_files:
        zf.write(f)

print(f"\n📦 Downloading {len(saved_files)} clips...")
files.download(zip_name)

## 🎛️ Tuning

```python
# More trains = more chaos
game = HeadlessGame(num_trains=15)

# Longer simulation = more events  
all_frames = extractor.run_simulation(total_frames=FPS * 300)  # 5 min

# More clips
clips = extractor.extract_clips(all_frames, num_clips=10)
```

### Strafe Detection
Clips are **rejected** if the killer:
- Changes direction >10 times/second (spinning)
- Moves <8 cells net displacement over 1 second (stuck/coiling)

Only clips where the killer keeps moving forward with purpose are kept!